<a href="https://colab.research.google.com/github/GabeMaldonado/AIforMedicine/blob/master/AIforMed_W3_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing the U-Net Architecture Using Keras

U-Nets are used for image segmentation. This architecture features a series of down-convolutions connected by max-pooling operations followed by a series of up-convolutions conected by upsampling and concatenation operations. Each down-convolution is also connected directly by the concatenation operation in the up-sampling portion of the network. 

## U-Net Diagram

![U-Net diagram. Source: Wikipedia.](https://upload.wikimedia.org/wikipedia/commons/2/2b/Example_architecture_of_U-Net_for_producing_k_256-by-256_image_masks_for_a_256-by-256_RGB_image.png)








In [1]:
# import required libraries to build U-Net

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras import backend as K 
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam
from keras.layers.merge import concatenate 

# Set the image shape to have channels in first dimension

K.set_image_data_format("channels_first")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
import tensorflow as tf

## Depth of the U-Net

The depth of the U-Net is determined by the number of down-convolutions in the architecture. For isntance, the diagram of the U-Net above has a depth of four because it has 4 down-convolutions including the one at the bottom of the U. 
In this lab, the depth of the U-Net will be two as we will declare two down-convolutions. Here we will be doing image segmentation so in addition to *height* and *width* the input later would also have *length*. Here we are deliberately using the work *length* to describe the third spatial dimension of the input so we do not confuse it with the *depth* of the U-Net. 
The sahpe of the input layer is ```(num_channels, height, width, length)``` where ```num_channels``` can be thought of as as the color channels in an image, ```height```, ```width```, and ```lenght``` are just the size of the input.
The value for these parameter are:
*   ```num_channels = 4```
*   ```height = 160```
*   ```width = 160```
*   ```length = 16```



In [3]:
# Define an input layer tensor of the shape show above

input_layer = Input(shape=(4, 160, 160, 16))
input_layer

<tf.Tensor 'input_1:0' shape=(None, 4, 160, 160, 16) dtype=float32>

Notice that we have a **?** as the very first dimensional. The first dimension is ```batch_size```. The dimension of a tensor are:

```(batch_size, num_channels, height, widtg  length)```

## Contracting / Downward Path

Here we will begin the construction of the downward path of network (the left side of the U-Net). The ```(height, width, length)``` of the input will get smaller as it moves down the path and the number of channels will increase. 

### Depth 0

By *depth 0* here, we are referring to the depth of the first down-convolution in the U-Net. 
The number of filters is specified for each depth and for each layer within that depth. 

The formula to calulate the number of filter is: 
$$filters_i = 32 \times (2^i)$$
Where $i$ is the current depth. 
So at depth $i = 0$
$$filters_0 = 32 \times (2^0) = 32 $$

### Layer 0
There are two convolutional layers at each depth.

In [0]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

In [5]:
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)


tf.__version__ is 2.2.0-rc4
tf.keras.__version__ is: 2.3.0-tf


In [0]:
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

In [7]:
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()

[]

In [8]:
# Define the 1st Conv3D layer/tensor with 32 filters

down_depth_0_layer_0 = Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1, 1, 1))(input_layer)
down_depth_0_layer_0

<tf.Tensor 'conv3d_1/add:0' shape=(None, 32, 160, 160, 16) dtype=float32>

In [9]:
# Add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<tf.Tensor 'activation_1/Relu:0' shape=(None, 32, 160, 160, 16) dtype=float32>

### Depth 0 Layer 1

For layer 1 of depth 0, the formula for calculating the number of filters is:

$$ filters_{i} = 32 \times (2^{i}) \times 2 $$

Where $i$ is the current depth.

Notice the $\times 2$ at the end of the formula is not there for layer 0.

So at depth $i$ = 0 for layer 1 =
$$filters_0 = 32 \times(2^0)\times 2 = 64$$

In [10]:
# Create a Conv layer with 64 filters and a relu activation function

down_depth_0_layer_1 = Conv3D(filters=64,
                              kernel_size=(3, 3, 3),
                              padding='same',
                              strides=(1, 1, 1))(down_depth_0_layer_0)
                    
# Add activation function
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

### MaxPooling
In the U-Net architecture, as seen in the image, there is a MaxPooling operation after each down-convolution excluding the last convolution at the bottom of the U. So the number of maxpooling operations would be $depth - 1$. 
Our example here is of depth 2 so the number of maxpooling operations is $2-1=1$ 


In [11]:
# Define a maxpooling layer

down_depth_0_layer_pool = MaxPooling3D(pool_size=(2, 2, 2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<tf.Tensor 'max_pooling3d_1/transpose_1:0' shape=(None, 64, 80, 80, 8) dtype=float32>

### Depth 1 Layer 0

To calculate the filters for depth 1 layer 0:

$$filters_1 = 32 \times(2^1) = 64$$

In [12]:
# Create a Conv3D layer to the newtwork with a relu activation

down_depth_1_layer_0 = Conv3D(filters=64,
                              kernel_size=(3, 3, 3),
                              padding='same',
                              strides=(1, 1, 1))(down_depth_0_layer_pool)

down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<tf.Tensor 'activation_3/Relu:0' shape=(None, 64, 80, 80, 8) dtype=float32>

### Depth 1 Layer 1

To calculate the filters for depth 1 layer 1:

$$filters_1 = 32\times(2^1)\times2= 128$$

In [13]:
# Create a Conv3D layer with 128 filter to the network

down_depth_1_layer_1 = Conv3D(filters=128,
                              kernel_size=(3, 3, 3),
                              padding='same',
                              strides=(1, 1, 1))(down_depth_1_layer_0)

down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<tf.Tensor 'activation_4/Relu:0' shape=(None, 128, 80, 80, 8) dtype=float32>

### MaxPooling

We reached the bottom of the **U** of depth 2 so no maxpooling is required after this down-convolution. 

### Depth 0 Upsampling Layer 0

We are now moving upward. For upsampling, we'll use a pool size of (2, 2, 2) as it is the default value in keras. 
The input to the first upsampling layer is the last layer of downsampling. In this example it is ```down_depth_1_layer_1```



In [14]:
# Create the upsampling layer to the network

up_depth_0_layer_0 = UpSampling3D(size=(2, 2, 2))(down_depth_1_layer_1)
up_depth_0_layer_0

<tf.Tensor 'up_sampling3d_1/concat_2:0' shape=(None, 128, 160, 160, 16) dtype=float32>

### Concatenation Operation

We must now concatenate the layers that are both located at 0 depth. 

*   ```up_depth_0_layer``` is of shape (None, 128, 160, 160, 16)
*   ```down_depth_layer_0_layer_1``` is of shape (None, 64, 160, 160, 16)
*    Ensure that these layers have the same height, width and length
*   If their shapes are the same, they can be concatenated along axis 1 (channel axis)

The height, width, and length is (160, 160, 16) for both-- but let's double-check:



In [15]:
# Check shapes of layers to concatenate

print(up_depth_0_layer_0)
print("--------")
print(down_depth_0_layer_1)

Tensor("up_sampling3d_1/concat_2:0", shape=(None, 128, 160, 160, 16), dtype=float32)
--------
Tensor("activation_2/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)


In [16]:
# Add concatenation along axis 1

up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)

up_depth_1_concat

<tf.Tensor 'concatenate_1/concat:0' shape=(None, 192, 160, 160, 16) dtype=float32>

The upsampling layer now has 192 channels: $128+64=192$ channels.

### Up-convolution Layer 1

The number of filters for this layer will be set for the number of channels in the down-convolution's layer 1 at the same depth of 0 (```down_depth_0_layer_1```)

In [17]:
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

In [18]:
# print number of filters
print(f"Number of filters: {down_depth_0_layer_1._keras_shape[1]}")

Number of filters: 64


In [19]:
# Create a Conv3D up-convolution layer with 64 filters

up_depth_1_layer_1 = Conv3D(filters=64,
                            kernel_size=(3, 3, 3),
                            padding='same',
                            strides=(1, 1, 1))(up_depth_1_concat)

up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<tf.Tensor 'activation_5/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

### Up-convolution Depth 0 Layer 2

At layer 2 of depth 0  the up-convolution the next step is to add another up-convolution. The number of filters for this layer is the same as the number of filters in the down-convolution depth 0 layer 1. 
Let's confirm using same method as above:


In [20]:
print (down_depth_0_layer_1)
print(f"Number of filters: {down_depth_0_layer_1._keras_shape[1]}")

Tensor("activation_2/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)
Number of filters: 64


In [0]:
# Create a Conv3D up-convolution layer with 64 filters

up_depth_1_layer_2 = Conv3D(filters=64,
                            kernel_size=(3, 3, 3),
                            padding='same',
                            strides=(1, 1, 1))(up_depth_1_layer_1)

up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)

In [22]:
up_depth_1_layer_2

<tf.Tensor 'activation_6/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

### Final Convolution

For the last convolution layer, we will set the number of filters to be equal to the number of classes in your input data. 
We are trying to classify for 3 classes:
1.   Edema
2.   Non-enhancing tumor
3.   Enhancing-tumor

In [23]:
# Add Final Conv3D layer with 3 filters to the network

final_conv = Conv3D(filters=3,
                   kernel_size=(1, 1, 1),
                   padding='valid',
                   strides=(1, 1, 1))(up_depth_1_layer_2)

final_conv

<tf.Tensor 'conv3d_7/add:0' shape=(None, 3, 160, 160, 16) dtype=float32>

In [24]:
# Add sigmoid activation to final conv

final_activation = Activation('sigmoid')(final_conv)
final_activation

<tf.Tensor 'activation_7/Sigmoid:0' shape=(None, 3, 160, 160, 16) dtype=float32>

### Create and Compile the Model

Here we will be seeting the loss and metrics to pre-built options in keras. 

In [0]:
# Create and compile model

model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
              )

In [26]:
# Print out model's summary

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 160, 160, 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 160, 160, 55360       activation_1[0][0]               
____________________________________________________________________________________________